In [ ]:
'''Start here.
Populate the following fields with the relevant information.
These should match information from your gcloud DICOM store.
See the gcloud dicom guide for more information:
  https://cloud.google.com/healthcare-api/docs/how-tos/dicom.

Search the file for "Try here" for interactive sections.
'''

project_name = ''
location = '' # e.g. us-west1
dataset_id = ''
dicom_store_id = ''
study_uid = ''
series_uid = ''

# Supply magnifications present in your DICOM Series
# Strings must match what is present in magnfication.py exactly.
low_magnification_str = '0.625X' # e.g. 0.625X, 1.25X, 2.5X
medium_magnification_str = '10X' # e.g. 5X, 10X
high_magnification_str = '40X' # e.g. 20X, 40

In [ ]:
### Import Dependencies

from ez_wsi_dicomweb import dicom_slide
from ez_wsi_dicomweb import dicom_web_interface
from ez_wsi_dicomweb import magnification
from ez_wsi_dicomweb import patch_generator
from hcls_imaging_ml_toolkit import dicom_web
from hcls_imaging_ml_toolkit import dicom_path
from dataclasses import dataclass
from matplotlib import pyplot as plt
from math import floor, sqrt

In [ ]:
### Define Constants

series_path_str = (
    f'projects/{project_name}/locations/{location}'
    f'/datasets/{dataset_id}/dicomStores/{dicom_store_id}'
    f'/dicomWeb/studies/{study_uid}'
    f'/series/{series_uid}'
)
series_path = dicom_path.FromString(series_path_str)
low_magnification = magnification.Magnification.FromString(low_magnification_str)
medium_magnification = magnification.Magnification.FromString(medium_magnification_str)
high_magnification = magnification.Magnification.FromString(high_magnification_str)

In [ ]:
### Create Dicom Web Client

dicom_web_client = dicom_web.DicomWebClientImpl()

In [ ]:
### Create DICOM Web Interface

dwi = dicom_web_interface.DicomWebInterface(dicom_web_client)

In [ ]:
### Create DicomSlide

ds = dicom_slide.DicomSlide(
    dwi=dwi,
    path=series_path,
    enable_client_slide_frame_decompression = True
    )

In [ ]:
### Using DicomSlide

# Print all the Magnifications in the DICOM series.
# An unknown magnification is one that is not mapped in magnification.py.
print(
    'Magnifications in'
    f' series:\n{[magnification.as_string for magnification in ds.magnifications]}'
)
# Get the entire image at a low magnification
small_image = ds.get_image(low_magnification).image_bytes()

# Image rendering code
plt.title(f'Magnification {low_magnification.as_string}')
plt.imshow(small_image)
plt.show()


In [ ]:
###  Extract a patch of a tissue sample.

# Try here. Set these coordinates for your patch
x_origin=100
y_origin=100
width=100
height=100

tissue_patch = ds.get_patch(
    low_magnification, x=x_origin, y=y_origin, width=width, height=height
).image_bytes()

# Image rendering code
plt.title(f'Patch at Magnification {low_magnification.as_string}')
plt.imshow(tissue_patch)
plt.show()

In [ ]:
### Boilerplate code for zooming in

@dataclass(frozen=True)
class PatchCoordinate:
  x_origin: int
  y_origin: int
  width: int
  height: int
  selected_magnification: magnification.Magnification

# Try here. Add magnifications for zoom in and set coordinates.
x_origin = 100
y_origin = 100
width = 100
height = 100
magnifications = [
    low_magnification,
    magnification.Magnification.FromString('2.5X'),
    medium_magnification,
    high_magnification,
]

coordinates = []
for higher_magnification in magnifications:
  # Scaling factor helps us determine the zoomed in coordinates, could use height instead to compute it.
  scaling_factor = higher_magnification.as_double / low_magnification.as_double
  coordinates.append(
      PatchCoordinate(
          x_origin=floor(x_origin * scaling_factor),
          y_origin=floor(y_origin * scaling_factor),
          width=floor(width * scaling_factor),
          height=floor(height * scaling_factor),
          selected_magnification=higher_magnification,
      )
  )


In [ ]:
### Patch fetching and image rendering code for zoom in

for i, coordinate in enumerate(coordinates):
  interesting_region_zoomed_in = ds.get_patch(
      coordinate.selected_magnification,
      x=coordinate.x_origin,
      y=coordinate.y_origin,
      width=coordinate.width,
      height=coordinate.height,
  ).image_bytes()

  # Image rendering code
  plt.subplot(1, len(magnifications), i + 1)
  plt.title(f'Magnification {coordinate.selected_magnification.as_string}')
  plt.imshow(interesting_region_zoomed_in)


plt.tight_layout()
plt.show()


In [ ]:
### Create a Patch Generator

# Try here. Play around with stride and patch size and the magnifications.
stride_size = 10
patch_size = 100
patch_magnification = medium_magnification
mask_magnification = low_magnification  # Used to approximate where tissue is.

patch_gen = patch_generator.PatchGenerator(
    slide=ds,
    stride_size=stride_size,
    patch_size=patch_size,
    magnification=patch_magnification,
    max_luminance=0.8,  # defaults to .8, heuristic to evaluate where tissue is.
    tissue_mask_magnification=mask_magnification,
)

In [ ]:
### Using Patch Generator

# Try here. Adjust the number of patches rendered. Use a square number.
n = 9
plt.figure(figsize=(patch_size / 5, patch_size / 5))  # Tune the plot sizes.

rows = floor(sqrt(n))
cols = rows
for i, patch in enumerate(patch_gen):
  plt.subplot(rows, cols, i + 1)
  plt.imshow(patch.image_bytes())
  if i >= n:
    break

plt.tight_layout()
plt.show()